# Выбор региона для нефтедобычи

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-модели-и-предсказание" data-toc-modified-id="Обучение-модели-и-предсказание-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение модели и предсказание</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-выбор-региона" data-toc-modified-id="Расчёт-прибыли-и-выбор-региона-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчёт прибыли и выбор региона</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

## Описание проекта

По условиям проекта мы работаем в добывающей компании. Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. В рамках проекта необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Получив предсказания модели мы должны рассчитать возможную прибыль и риски техникой *Bootstrap.*

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

## Загрузка и подготовка данных

Начнем с того, что загрузим данные и подготовим их к обучению модели.

In [2]:
try:
    geo_data_0 = pd.read_csv('geo_data_0.csv')
    geo_data_1 = pd.read_csv('geo_data_1.csv')
    geo_data_2 = pd.read_csv('geo_data_2.csv')
except:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
names = ['Регион 1', 'Регион 2', 'Регион 3']

for name, region in zip(names, [geo_data_0, geo_data_1, geo_data_2]):
    print(name)
    print()
    region.info()
    print()

Регион 1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Регион 2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Регион 3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64

Как видим, пропусков нет ни в одном столбце. Заполнять и преобразовывать ничего не нужно. Поскольку мы не знаем, что собой представляют значения в столбцах f0, f1, f2, то их распределение нам тоже особо ничего не скажет. Посмотрим, как выглядят наши данные в таблице.

In [4]:
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Видим столбец под название id. Для обучения он нам не пригодится, можно столбец удалить.

In [5]:
geo_data_0 = geo_data_0.drop('id', axis=1)

Также поступим и с остальными двумя таблицами.

In [6]:
geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
geo_data_1 = geo_data_1.drop('id', axis=1)

In [8]:
geo_data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
geo_data_2 = geo_data_2.drop('id', axis=1)

Итак, мы ознакомились с данными и внесли некоторые коррективы. Далее мы приступим к обучению и посмотрим, как линейная регрессия справляется с полученными нами данными.

## Обучение модели и предсказание

В этом шаге обучим и проверим модель для каждого региона.

Для начала разобъем данные на обучающую и валидационную выборки в соотношении 75:25. Так как нами будет использоваться алгоритм линейной регресии, чувствительный к выбросам, полученные фичи мы приведем к одному масштабу.
После этого обучим модель и сделаем предсказания на валидационной выборке. В качестве метрики качества предсказаня модели будем использовать RMSE.

Предсказания и целевой признак мы будем хранить в двух отдельных структурах, к которым в последствии будем обращаться.

In [10]:
predictions = pd.DataFrame() # создаем структуру, в которой будут храниться предсказания
target_values = pd.DataFrame() # здесь будут храниться правильные ответы

for name, region in zip(names, [geo_data_0, geo_data_1, geo_data_2]):
    features = region.drop('product', axis=1)
    target = region['product']

    features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                                   target, test_size = 0.25, random_state=12345)
    
    target_values[name] = target_valid
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    model = LinearRegression()
    model.fit(features_train, target_train)

    model_prediction = pd.Series(model.predict(features_valid), index=target_valid.index)
    rmse = mean_squared_error(target_valid, model_prediction) ** 0.5
    
    predictions[name] = model_prediction

    print(name)
    print("Среднее предсказаний:", model_prediction.mean())
    print("Реальное среднее:", target_valid.mean())
    print('RMSE:', rmse)
    print()

Регион 1
Среднее предсказаний: 92.59256778438038
Реальное среднее: 92.07859674082927
RMSE: 37.5794217150813

Регион 2
Среднее предсказаний: 68.728546895446
Реальное среднее: 68.72313602435997
RMSE: 0.8930992867756158

Регион 3
Среднее предсказаний: 94.96504596800489
Реальное среднее: 94.88423280885438
RMSE: 40.02970873393434



Пройдемся по регионам:

**Регион 1**

Мы получили предсказания запаса сырья, со средним очень близким к среднему значению реальных данных. Однако RMSE модели получился достаточно большим — всего в 2,5 раза меньше среднего значения.

**Регион 2**

В данном регионе результат RMSE заметно лучше — меньше единицы. Среднее значение также почти совпадает с реальным средним значением запасов сырья. Тем не менее, среднее значение в этом регионе существенно меньше, чем в предыдущем.

**Регион 3**

Ситуация в этом регионе схожа с ситуацией в первом — среднее значение больше, чем во втором, но ошибка RMSE большая.

В ходе обучения модели линейной регрессии, проведенного на данных трех регионах, мы выявили, что модель хорошо справилась с предсказаниями на втором регионе, но значительно хуже на первом и третьем. Вероятно, данные в этих двух регионах имеют большие колебания и меньше поддаются логике, встроенной в линейную модель.

## Подготовка к расчёту прибыли

На этом шаге подготовимся к расчёту прибыли и сохраним все ключевые значения для расчётов отдельных переменных.

In [11]:
points = 500 # при разведке региона исследуют 500 точек,
top_points = 200 # из которых выбирают 200 лучших для разработки
budget = 10000000000 # бюджет на разработку скважин в регионе
price_per_unit = 450000 # доход с каждой единицы продукта — 450 тыс. рублей

Теперь рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. После чего сравним полученный объём сырья со средним запасом в каждом регионе.

In [12]:
no_loss_dev = budget / price_per_unit / top_points
no_loss_dev

111.11111111111111

Итак, рассчитанный достаточный объем сырья превышает средний запас в каждом регионе. В случае со вторым регионом, разница примерно в 2 раза. Можно ли уже сделать вывод, что второй регион точно не позволит нам заработать на разработке скважин? Мы предполагаем, что это сравнение не слишком репрезентативно, поскольку при расчете среднего запаса были использованы значения по 25 тыс. точек, включая точки с наименьшим запасом. В разработке такие участвовать не будут. Поэтому в следующем шаге мы посмотрим, какую примерно прибыль мы сможем получить, если будем выбирать только наилучшие точки для разработки.

## Расчёт прибыли и выбор региона

В этом шаге мы постараемся рассчитать предполагаемую прибыль по каждому их регионов и высчитать риски убытков по каждому из них. Начнем с того, что напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [13]:
def calculate_profit(predictions, target, top_points, price_per_unit, budget):
    top_200 = predictions.sort_values(ascending=False)[:top_points] # скважины с максимальными значениями предсказаний
    target_sum = target[top_200.index].sum() # сумма целевого значения объёма сырья 
    return (target_sum * price_per_unit) - budget # прибыль для полученного объёма сырья

Теперь посчитаем риски и прибыль для каждого региона с помощью техники Bootstrap с 1000 выборок и найдем распределение прибыли. Также найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков (отрицательной прибыли).

In [14]:
state = RandomState(12345)

In [15]:
for name, region in zip(names, [geo_data_0, geo_data_1, geo_data_2]):
    
    profits = [] # здесь будут храниться все значения прибыли
    
    for i in range(1000):
        subsample = predictions[name].sample(n=points, replace=True, random_state=state)
        profit = calculate_profit(subsample, target_values[name], top_points, price_per_unit, budget)
        profits.append(profit)

    profits = pd.Series(profits)
    
    mean_profit = profits.mean()
    risk = len([profit for profit in profits if profit < 0]) / len(profits)
    confidence_interval = st.t.interval(0.95, df=len(profits)-1, loc=profits.mean(), scale=profits.sem())

    print(name)
    print("95%-ый доверительный интервал:", confidence_interval)
    print('Средняя прибыль:', mean_profit)
    print('Риск убытков в %:', risk*100)
    print()

Регион 1
95%-ый доверительный интервал: (379620315.1479725, 412709654.45676965)
Средняя прибыль: 396164984.8023711
Риск убытков в %: 6.9

Регион 2
95%-ый доверительный интервал: (448828936.4815786, 473482698.0729009)
Средняя прибыль: 461155817.27723974
Риск убытков в %: 0.7000000000000001

Регион 3
95%-ый доверительный интервал: (376164225.32954943, 409736725.0116595)
Средняя прибыль: 392950475.17060447
Риск убытков в %: 6.5



Пройдемся по регионам:

**Регион 1**

Получается, что в среднем в этом регионе мы можем заработать около 400 млн. рублей, однако риск убытков составляет почти 7%, а по условиям нашего проекта нам необходимо добиться результата, не превышающего 2,5%.

**Регион 2**

В этом регионе средняя прибыль ближе к 450 млн. рублей, а риск убытков ниже 1%. В целом, данный регион подходит под условия, установленные бизнесом, осталось посмотреть лишь на третий регион.

**Регион 3**

В этом регионе средняя прибыль меньше, чем в предыдущих двух, а риски убытков превышают 6%. Регион тоже можно отбросить.

### Вывод

Мы провели исследование трех регионов для выяснения, какой из них может оказаться наиболее прибыльным при разработке новых нефтяных скважин. Для получения такого прогноза мы обучили модель линейной регрессии на данных этих трех регионов. В результате исследования мы выявили, что наиболее прибыльной разработка скважин может быть в регионе №2, поскольку средняя прибыль по нему превысила средние показатели других регионов. Также для каждого региона был рассчитан риск убытков, и оказалось, что именно в этом регионе риск является наименьшим и не превышает установленного бизнесом критерия в 2,5%.